### 데이터 수집
    * 스크래이핑을 통한 수집
        - 뉴스
        - SNS
        - OTT
        - 댓글
    * json api를 활용
        - 핀테크 금융
        - 프롭테크 부동산
        - 마케팅
        - SCM
        - CRM
    * 빅데이터 생성
    * DB 활용
    * 시각화
    * 사무 자동화
        - 오피스
        - RPA
    * 사회소통능력 
        - 유튜브
        - SNS
        -홈페이지 관리
### 개별 데이터 분석
    * 정형 데이터 분석: 개별분석
        - 모멘텀 분석
        - 기초 분석
    * 비정형 데이터 분석: 개별분석
        - Count Vectorize
        - TF-IDF
        - Word2Vec:cbow
        - LDA

### 데이터 융합
    * 정형데이터: 주가 데이터
    * 비정형 데이터: 뉴스 데이터
    * 데이터 융합
        - 데이터베이스 융합: DB/SQL join
        - Pandas 활용: Merge 명령어 사용
### 고급 데이터 처리 기법
*  해당분야 용어 기법
* 

In [1]:
### 기초데이터: DB join 방식 활용
### count 벡터화: Pandas 활용
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

In [2]:
dbname='./navernews.db'
def getDB(sql='select nidx,ndate from nnews where nday is NULL'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #sql='select * from stockp where skey>20200101 order by skey asc'
        qry=cur.execute(sql)
        desc=[row[0] for row in qry.description]
        data=qry.fetchall()
    return((desc,data))

In [3]:
def dbNday(ndays):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #sql='select * from stockp where skey>20200101 order by skey asc'
        sql='update  nnews set nday="'+ndays[1]+'" where nidx = '+str(ndays[0])
        qry=cur.execute(sql)
    return(qry)

In [4]:
def updateNdays():
    res=getDB()
    ndays=[]
    for r in res[1]:
        ndays.append((r[0],r[1][:10]))
    for n in ndays:
        print(dbNday(n))

In [6]:
updateNdays()

In [8]:
sql= "select sday,sopen,shigh,slow,sclose,svol,count(nidx) nd,group_concat(ntitle) gtitle from stockp a left outer join nnews b on a.sday=b.nday group by sday"
mdata=getDB(sql)

In [9]:
df=pd.DataFrame(data=mdata[1],columns=mdata[0])
df.tail()

,sday,sopen,shigh,slow,sclose,svol,nd,gtitle
885,2021-03-29,282000,282000,276000,277000,53284,0,None
886,2021-03-30,276500,279500,276000,279000,41692,0,None
887,2021-03-31,279000,281500,276500,281000,58150,0,None
888,2021-04-01,282500,283000,277500,279000,72605,0,None
889,2021-04-02,278000,278500,274000,274500,55290,0,None


In [53]:
nlpdf=df[['sday','gtitle']].copy()
nf=nlpdf.fillna('')
nf.head()

,sday,gtitle
0,2017-08-16,
1,2017-08-17,
2,2017-08-18,
3,2017-08-21,
4,2017-08-22,


In [46]:
from konlpy.tag import Kkma
kkma=Kkma()

def dictsort(r):
    sr=sorted(r.items(),key=(lambda x:x[1]),reverse=True)
    return r

def getPOS(rres,limlen=0):
    p=kkma.pos(rres)
    dictpos={}
    words=[]
    #표제어 추출
    check=['NNG','NNP','NNB','NNM','VV','VA','VXV','VXA','VCP','VCN','SF','EFN','EFQ']
    for  w,pos in p:
        if(pos in check):
            if(len(w)>limlen):
                words.append(w)
                if (w in dictpos.keys()):
                    dictpos[w]+=1
                else:
                    dictpos[w]=1
    return (dictpos,words)  

In [60]:
gtitles=nf.values
gtitles[-200:-195]

array([['2020-06-12', ''],
       ['2020-06-15', ''],
       ['2020-06-16', ''],
       ['2020-06-17',
        ' 포토  신세계  푸빌라 캐릭터 상품, 포토  신세계 푸빌라 비치백  비치타올, 포토  신세계백  푸빌라 캐릭터 상품 선보여  , 포토  신세계백화점  푸빌라 캐릭터'],
       ['2020-06-18', '신세계百  휴가철 앞두고 대형  와인 장터  연다']], dtype=object)

In [68]:
corpus=[]
for g in gtitles:
    res=getPOS(g[1],1)
    corpus.append(' '.join(res[1]))

In [78]:
# 카운트 기반 TDM
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer(min_df=2)
tdm=vect.fit_transform(corpus).toarray()
tdm.shape

(890, 1998)

In [79]:
gtitles.shape

(890, 2)

In [70]:
tdm[-200:-195]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [96]:
tdms=[]
for i,t in enumerate(tdm):
    res=list(t)
    res.insert(0,gtitles[i][0])
    tdms.append(res)

In [100]:
tdmdf=pd.DataFrame(data=tdms)
tdmdf.head()

,0,1,2,3,4,5,6,7,8,9,...,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998
0,2017-08-16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2017-08-17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017-08-18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017-08-21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2017-08-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
